# N8N Workflow Generator - Inference Notebook

Use the trained LoRA model with Mistral-7B to generate n8n workflows from natural language descriptions.

## Model Information:
- **Base Model**: mistralai/Mistral-7B-Instruct-v0.2
- **Training Method**: LoRA (Low-Rank Adaptation)
- **LoRA Rank**: 16
- **Dataset**: n8n workflow examples with natural language descriptions
- **Training Status**: ✅ Successfully Trained
- **Model Location**: `./trained_model/`

## Step 1: Install Required Packages

In [ ]:
# Install required packages (this may take a few minutes)
!pip install -q transformers peft accelerate bitsandbytes torch
print('✅ Packages installed successfully!')

## Step 2: Import and Setup

In [ ]:
import torch
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

print('✅ All imports successful!')
print(f'PyTorch version: {torch.__version__}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## Step 3: Load the Trained Model

In [ ]:
# Path to the trained model
model_dir = Path('./trained_model')
base_model = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"🔧 Loading model from {model_dir}...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
print("✅ Tokenizer loaded")

# Load base model with 4-bit quantization
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        device_map="auto"
    )
    print("✅ Base model loaded with 4-bit quantization")
except Exception as e:
    print(f"⚠️ Quantization failed: {e}")
    print("Loading with fp16 instead...")
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("✅ Base model loaded with fp16")

# Load LoRA adapter
model = PeftModel.from_pretrained(model, model_dir)
model.eval()
print("✅ LoRA adapter loaded")
print("\n✅ Model fully loaded and ready for inference!")

## Step 4: Generate n8n Workflows

In [ ]:
def generate_workflow(prompt, max_length=2048, temperature=0.7, top_p=0.9):
    """
    Generate an n8n workflow from natural language description
    
    Args:
        prompt: Natural language description
        max_length: Maximum tokens to generate
        temperature: Sampling temperature (higher = more creative)
        top_p: Nucleus sampling parameter
    
    Returns:
        Generated workflow (text)
    """
    # Format prompt in training style
    formatted_prompt = f"""<|system|>
You are an n8n workflow generator. Convert natural language descriptions into valid n8n workflow JSON.
<|user|>
{prompt}
<|assistant|>
"""
    
    # Tokenize
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract assistant response
    if "<|assistant|>" in response:
        response = response.split("<|assistant|>")[-1].strip()
    
    return response

print("✅ Generation function defined")

## Example 1: Webhook + Email Notification

In [ ]:
prompt1 = "Create a workflow that receives data from a webhook, processes it, and sends an email notification"

print(f"📝 Prompt: {prompt1}\n")
print("🔄 Generating workflow...\n")

workflow1 = generate_workflow(prompt1)
print("📋 Generated Workflow:")
print(workflow1)

## Example 2: API Data Collection

In [ ]:
prompt2 = "Build a workflow that fetches data from an API every hour and stores it in a database"

print(f"📝 Prompt: {prompt2}\n")
print("🔄 Generating workflow...\n")

workflow2 = generate_workflow(prompt2)
print("📋 Generated Workflow:")
print(workflow2)

## Example 3: Slack Message Logging

In [ ]:
prompt3 = "Generate a workflow that listens for Slack messages and logs them to a file"

print(f"📝 Prompt: {prompt3}\n")
print("🔄 Generating workflow...\n")

workflow3 = generate_workflow(prompt3)
print("📋 Generated Workflow:")
print(workflow3)

## Try Your Own Prompt

In [ ]:
# 🎯 Try your own workflow description here!
custom_prompt = "YOUR WORKFLOW DESCRIPTION HERE"

print(f"📝 Your Prompt: {custom_prompt}\n")
print("🔄 Generating workflow...\n")

custom_workflow = generate_workflow(custom_prompt)
print("📋 Generated Workflow:")
print(custom_workflow)

# Try to parse as JSON
try:
    json_start = custom_workflow.find('{')
    json_end = custom_workflow.rfind('}') + 1
    if json_start >= 0 and json_end > json_start:
        json_str = custom_workflow[json_start:json_end]
        workflow_json = json.loads(json_str)
        print("\n✅ Valid JSON detected:")
        print(json.dumps(workflow_json, indent=2))
except:
    print("\n⚠️ Could not parse JSON from response")

## Model Performance Notes:

- The model was trained on n8n workflow examples with LoRA (parameter-efficient fine-tuning)
- It uses a Mistral-7B instruction-tuned base model for better instruction following
- Generation quality can be improved by:
  - Adjusting `temperature` (lower = more deterministic, higher = more creative)
  - Adjusting `top_p` (nucleus sampling for diversity)
  - Providing more specific, detailed prompts
  - Post-processing the JSON output if needed